# Debugging of the 'forecast_regressor_construction' function 

In [11]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import concat

In [12]:
#Lettura del File
df=pd.read_excel('SHILLER_Data.xlsx')

#Impostazione della data come indice
df.set_index('Date', inplace=True)
df.head()

#Eliminazione delle colonne Dividends e CPI
df=df.drop(['Dividends' ,'CPI'],axis=1)
df.dropna()

,S&P 500,Earnings,10Y Usa Rate
Date,,,
1900-01-31,6.100000,0.480000,3.150000
1900-02-28,6.210000,0.480000,3.145833
1900-03-31,6.260000,0.480000,3.141667
1900-04-30,6.340000,0.480000,3.137500
1900-05-31,6.040000,0.480000,3.133333
...,...,...,...
2023-08-31,4457.358696,183.170000,4.170000
2023-09-30,4409.095000,184.250000,4.380000
2023-10-31,4269.400909,186.976667,4.800000


In [13]:
# Selezionde dati (dal 1945-01-31 in poi)
initial_date = '1945-01-31'
df = df[df.index >= initial_date]
df.head()

,S&P 500,Earnings,10Y Usa Rate
Date,,,
1945-01-31,13.49,0.940000,2.370
1945-02-28,13.94,0.950000,2.355
1945-03-31,13.93,0.960000,2.340
1945-04-30,14.28,0.973333,2.325
1945-05-31,14.82,0.986667,2.310


In [14]:
# Trasformazione 
l_df=np.log(df)

# Trasformazione in diff_log
d_l_df = l_df.diff().dropna()


In [15]:
d_l_df

,S&P 500,Earnings,10Y Usa Rate
Date,,,
1945-02-28,0.032814,0.010582,-0.006349
1945-03-31,-0.000718,0.010471,-0.006390
1945-04-30,0.024815,0.013793,-0.006431
1945-05-31,0.037118,0.013606,-0.006473
1945-06-30,0.018055,0.013423,-0.006515
...,...,...,...
2023-08-31,-0.011314,0.005914,0.066939
2023-09-30,-0.010887,0.005879,0.049133
2023-10-31,-0.032196,0.014690,0.091567


In [ ]:
def forecast_regressors_construction(d_l_PX, lag_ECM):
    # Lista per accumulare i valori
    Regressors_list = []

    n_vars = len(d_l_PX.columns)

    # Aggiungi a Regressors_list i valori delle ultime 'lag_ECM' righe
    for i in range(1, lag_ECM + 1):
        for j in range(n_vars):
            Regressors_list.append(d_l_PX.iloc[-i, j])
    
    # Aggiungi l'ultimo valore di residuals_Step1
   # Regressors_list.append(residuals_Step1.iloc[-1])

    # Converti la lista in un array NumPy
    Regressors = np.array(Regressors_list)
    # 1: l'array risultante avrà una sola riga
    #-1: l'array risultante avrà tante colonne quanti sono gli element dell'array originale
    Regressors = Regressors.reshape(1, -1)
                                
    return Regressors

In [17]:
d_l_df.tail()

,S&P 500,Earnings,10Y Usa Rate
Date,,,
2023-08-31,-0.011314,0.005914,0.066939
2023-09-30,-0.010887,0.005879,0.049133
2023-10-31,-0.032196,0.014690,0.091567
2023-11-30,0.043689,0.014478,-0.064539
2023-12-31,0.049214,0.014271,-0.112795


In [ ]:
argomento_forecast_1 = forecast_regressors_construction(d_l_df, 1)
print(argomento_forecast_1)

[[ 0.04921394  0.014271   -0.11279549]]


In [ ]:
argomento_forecast_2 = forecast_regressors_construction(d_l_df, 2)
print(argomento_forecast_2)

[[ 0.04921394  0.014271   -0.11279549  0.04368945  0.01447762 -0.06453852]]


In [ ]:
argomento_forecast_4 = forecast_regressors_construction(d_l_df, 4)
print(argomento_forecast_4)

[[ 0.04921394  0.014271   -0.11279549  0.04368945  0.01447762 -0.06453852
  -0.03219594  0.0146903   0.09156719 -0.01088692  0.00587885  0.04913269]]
